# Prompt Types Condition Entropy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import output_processing as op
from functools import reduce
from prompts import prompt_types

/Users/ab-purushothama-macbook/.conda/envs/prej-prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models= [
   "meta-llama/Llama-3.2-1B",
    "meta-llama/Llama-3.2-1B-Instruct",
    "meta-llama/Llama-3.2-3B",
    "meta-llama/Llama-3.2-3B-Instruct",
    "meta-llama/Llama-3.1-8B",
    "meta-llama/Llama-3.1-8B-Instruct",
    "gpt2-large",
    # "allenai/OLMo-2-0425-1B",
    # "allenai/OLMo-2-0425-1B-Instruct",
    # "allenai/OLMo-2-1124-7B",
    # "allenai/OLMo-2-1124-7B-Instruct",
    "ministral/Ministral-3b-instruct",
    "ministral/Ministral-4b-instruct",
    "mistralai/Ministral-8B-Instruct-2410",
    # "google/gemma-3-1b-it",
    # "google/gemma-3-4b-it",
    # "google/gemma-7b-it"
]
def read_and_organize_model_results(model_name):
    model_results = pd.read_csv(f"runs/runs-1001-05_19/{model_name}-results.csv")
    model_results.replace([0.0], -65504, inplace=True)
    model_results = op.organize_distribution(model_results)
    model_results["model_name"] = model_name.split("/")[-1]
    return model_results
    
combined_results = [read_and_organize_model_results(model_name) for model_name in models]
combined_results = pd.concat(combined_results).reset_index()
combined_results.shape

(12420, 17)

In [3]:
np.sum(combined_results[["B_probs"]] < 0, axis=0)

B_probs    0
dtype: int64

# We have sample level entropy

In [4]:
combined_results.sample()[["title", "model_name", "prompt_type", "entropy"]]

,title,model_name,prompt_type,entropy
9426,Hot Work II,Ministral-3b-instruct,disagreement,0.452462


# Conditional Entropy

<https://datascience.stackexchange.com/questions/58565/conditional-entropy-calculation-in-python-hyx>

<https://www.cs.cmu.edu/~venkatg/teaching/ITCS-spr2013/notes/lect-jan17.pdf>

In [5]:
relative_measures = op.calculate_relative_measures(combined_results)

In [6]:
relative_measures.shape

(11040, 6)

In [7]:
for group, df in combined_results.groupby(['title', 'version', 'model_name'], sort=False):
    control_mask = df.prompt_type == "yes_or_no"
    control_probs = df[["Aff_prob", "UnAff_prob", "Other_prob"]][control_mask]
    for variation in prompt_types[1:]:
        variation_mask = df.prompt_type == variation
        variation_probs = df[["Aff_prob", "UnAff_prob", "Other_prob"]][variation_mask]
        print(np.outer(control_probs.values, variation_probs.values))
        print(control_probs.shape, variation_probs.shape, op.conditional_entropy(control_probs.values, variation_probs.values))
        break
    break
            

[[0.01435467 0.01621104 0.09365418]
 [0.00974624 0.01100664 0.06358739]
 [0.09145765 0.10328508 0.59669711]]
(1, 3) (1, 3) 0.6407506743327896


In [9]:
relative_measures.sample()

,title,version,prompt_type,js_dist,more_than_reversal,cond_entropy
2086,Vehicle Fire III,controversial,options,0.0,0.0,0.275967
